In [9]:
import numpy as np
from scipy.stats import truncnorm, norm, beta, bernoulli,t


def simulate_data_2lc_t(N, I, pi_true, alpha_true, p_true, seed=None):
    """
    Fast vectorised version:
      • 输入参数与旧函数完全一致；
      • 返回 dict 的键/值类型也与旧版一致 (D,T,R 为 int64，T_obs 为 float64 带 NaN)。
    """
    rng = np.random.default_rng(seed)

    # 1) D_k ~ Bern(pi)
    D = rng.binomial(1, pi_true, size=N).astype(np.int64)        # (N,)

    # 2) T_{ki} | D_k
    #    α 对应列索引为 D；alpha_true shape (I,2)
    prob_T = alpha_true[:, D].T                                  # (N,I)
    T = (rng.random((N, I)) < prob_T).astype(np.int64)           # (N,I)

    # 3) R_{ki} | T_{ki}
    #    用布尔矩阵线性组合而非 take_along_axis
    prob_R = p_true[:, 0][None, :] * (1 - T) + p_true[:, 1][None, :] * T
    R = (rng.random((N, I)) < prob_R).astype(np.int64)           # (N,I)

    # 4) 观测 T
    T_obs = T.astype(float)
    T_obs[R == 0] = np.nan

    return {'D': D, 'T': T, 'R': R, 'T_obs': T_obs}







In [15]:
from tqdm import tqdm
from tqdm import trange
import numpy as np

import numpy as np
from tqdm import tqdm

def gibbs_sampler_2lc_t(T_obs, R,
                             n_iter=3000, burn=1000,
                             a_pi=1, b_pi=1,          # π 的先验
                             a0=1, b0=1, a1=1, b1=1,   # α 的先验
                             c0=1, d0=1, c1=1, d1=1,   # p 的先验 (t=0,1)
                             ident_order=True,
                             seed=None, verbose=True):
    """
    Gibbs sampler for 2LC-T with unknown p_{it}.
    返回 dict 抽样列表；其中 α, p 形状均为 (I,2)。
    """
    rng = np.random.default_rng(seed)

    N, I = T_obs.shape
    miss_mask = np.isnan(T_obs)

    # ---------- 初始化 ----------
    T = np.where(miss_mask, rng.binomial(1, .5, (N, I)), T_obs).astype(int)
    D = rng.binomial(1, .5, N)
    alpha = np.full((I, 2), .5)
    p     = np.full((I, 2), .8)            # 报告概率的初值，可自定义
    pi    = .5

    draws = []
    if verbose:
        pbar = tqdm(total=n_iter, desc="Gibbs", unit="iter", leave=True)

    for it in range(n_iter):
        # =========================================================
        # 1) 填补缺失 T | R=0, D, α, p
        # ---------------------------------------------------------
        alpha_mat = alpha[:, 0][None, :]*(1-D)[:, None] + alpha[:, 1][None, :]*D[:, None]
        p0, p1 = p[:, 0][None, :], p[:, 1][None, :]

        num = alpha_mat * (1-p1)              # α*(1-p1)
        den = num + (1-alpha_mat)*(1-p0)
        prob_mis = num / den
        T[miss_mask] = (rng.random(miss_mask.sum()) < prob_mis[miss_mask]).astype(int)

        # =========================================================
        # 2) 更新 D | T, α, π   （与原版相同）
        log_a1, log_1a1 = np.log(alpha[:,1])[None,:], np.log1p(-alpha[:,1])[None,:]
        log_a0, log_1a0 = np.log(alpha[:,0])[None,:], np.log1p(-alpha[:,0])[None,:]
        loglik1 = np.log(pi)      + (T*log_a1 + (1-T)*log_1a1).sum(axis=1)
        loglik0 = np.log1p(-pi)   + (T*log_a0 + (1-T)*log_1a0).sum(axis=1)
        prob_D1 = 1/(1+np.exp(loglik0-loglik1))
        D = (rng.random(N) < prob_D1).astype(int)

        # =========================================================
        # 3) π | D
        s1 = D.sum()
        pi = rng.beta(a_pi+s1, b_pi+N-s1)

        # =========================================================
        # 4) α | D, T
        n0, n1 = (D==0).sum(), s1
        succ0  = T[D==0].sum(axis=0)
        succ1  = T[D==1].sum(axis=0)
        alpha[:,0] = rng.beta(a0+succ0, b0+n0-succ0)
        alpha[:,1] = rng.beta(a1+succ1, b1+n1-succ1)

        if ident_order:
            swap = alpha[:,1] < alpha[:,0]
            alpha[swap] = alpha[swap][:, ::-1]

        # =========================================================
        # 5) p | T, R
        #    对每个检验 i、结果 t 独立 Beta 更新
        #    成功 = 报告且 T=t；失败 = 未报告且 T=t
        for t, (c_t, d_t) in enumerate([(c0,d0), (c1,d1)]):
            mask = (T == t)
            succ = (R & mask).sum(axis=0)            # R=1 & T=t
            tot  = mask.sum(axis=0)
            p[:, t] = rng.beta(c_t + succ, d_t + tot - succ)

        # =========================================================
        # 保存
        if it >= burn:
            draws.append(dict(alpha=alpha.copy(),
                              p=p.copy(),
                              pi=pi))

        if verbose and it % 100 == 0:
            pbar.update(100)
            pbar.set_description(f"π={pi:.4f}")

    return draws        # list 长度 = n_iter-burn


p_true = np.array([[0.9, 0.8], [0.9, 0.7], [0.8, 0.65], [0.8, 0.6], [0.9, 0.7]]) 
pi = 0.4
alpha_true = np.array([[0.6, 0.8], [0.3, 0.9], [0.4, 0.7], [0.5, 0.9], [0.2, 0.9]])
N=5000
I=5

data = simulate_data_2lc_t(N,I,pi_true = pi,alpha_true = alpha_true,p_true = p_true,seed = 42)
T_obs = data['T_obs']
print(T_obs)
print(data['R'])

samples = gibbs_sampler_2lc_t(
    T_obs=data['T_obs'],
    R=data['R'],
    n_iter=3000, burn=1000,
    seed=42
)
print(np.mean([sample['p'] for sample in samples], axis=0))


[[ 1.  1. nan  1.  1.]
 [ 1.  0.  0.  0.  0.]
 [ 1.  1. nan  1.  1.]
 ...
 [ 1. nan  1. nan  0.]
 [ 1.  0.  0.  1.  0.]
 [ 1.  1. nan nan  1.]]
[[1 1 0 1 1]
 [1 1 1 1 1]
 [1 1 0 1 1]
 ...
 [1 0 1 0 1]
 [1 1 1 1 1]
 [1 1 0 0 1]]


Gibbs:   0%|          | 0/3000 [00:00<?, ?iter/s]

π=0.4111: 100%|██████████| 3000/3000 [00:09<00:00, 306.94iter/s] 

[[0.86390788 0.82653907]
 [0.91076668 0.67047922]
 [0.80309582 0.65582985]
 [0.78840667 0.60259862]
 [0.91797405 0.6810159 ]]


In [16]:
import numpy as np
from tqdm import trange

# ------- 1. 单次模拟并返回后验 draws -------
def run_once(N, I, pi, alpha_true, p_true,
             n_iter=3000, burn=1000, seed=None):
    data = simulate_data_2lc_t(N, I, pi, alpha_true, p_true, seed)
    draws = gibbs_sampler_2lc_t(data['T_obs'], data['R'], 
                                n_iter=n_iter, burn=burn, seed=seed,verbose=False)

    # draws['alpha'] 形状: (n_save, I, 2)
    alpha_draws = np.stack([d['alpha'] for d in draws], axis=0)

    # 后验均值
    alpha_hat = alpha_draws.mean(axis=0)          # (I,2)

    # 95% 等尾区间
    ci_low  = np.quantile(alpha_draws, 0.025, axis=0)
    ci_high = np.quantile(alpha_draws, 0.975, axis=0)

    return dict(
        alpha_hat=alpha_hat,
        ci_low=ci_low,
        ci_high=ci_high,
        alpha_draws=alpha_draws        # 可选：若存储太大可删去
    )

# ------- 2. Monte-Carlo 主循环 -------
def run_mc(M=200,                        # 模拟重复次数
           N=1000, I=5,
           pi=0.4, alpha_true=None, p_true=None,
           n_iter=3000, burn=1000, seed0=123):

    if alpha_true is None or p_true is None:
        raise ValueError("请提供 alpha_true 与 p_true")

    rng = np.random.default_rng(seed0)

    # 结果容器
    cover_se  = np.zeros((M, I))
    cover_sp  = np.zeros((M, I))
    bias_alpha = np.zeros((M, I, 2))
    mse_alpha  = np.zeros((M, I, 2))

    for m in trange(M, desc="Monte-Carlo"):
        seed = int(rng.integers(1e9))
        res  = run_once(N, I, pi, alpha_true, p_true,
                        n_iter=n_iter, burn=burn, seed=seed)

        alpha_hat = res['alpha_hat']
        ci_low, ci_high = res['ci_low'], res['ci_high']

        # -------- se/sp 覆盖率 --------
        se_true = alpha_true[:, 1]
        sp_true = 1 - alpha_true[:, 0]

        se_hat_low, se_hat_high = ci_low[:, 1],  ci_high[:, 1]
        sp_hat_low, sp_hat_high = 1 - ci_high[:, 0], 1 - ci_low[:, 0]  # 注意顺序

        cover_se[m] = (se_true >= se_hat_low) & (se_true <= se_hat_high)
        cover_sp[m] = (sp_true >= sp_hat_low) & (sp_true <= sp_hat_high)

        # -------- α 偏差 / MSE --------
        bias_alpha[m] = alpha_hat - alpha_true          # (I,2)
        mse_alpha[m]  = (alpha_hat - alpha_true)**2

    # ========= 汇总 =========
    result = {
        "coverage_se":  cover_se.mean(axis=0),          # (I,)
        "coverage_sp":  cover_sp.mean(axis=0),
        "bias_alpha":   bias_alpha.mean(axis=0),        # (I,2)
        "mse_alpha":    mse_alpha.mean(axis=0)
    }
    return result

# ----------------- 运行示例 -----------------
p_true = np.array([[0.9, 0.8],
                   [0.9, 0.7],
                   [0.8, 0.65],
                   [0.8, 0.6],
                   [0.9, 0.7]])
alpha_true = np.array([[0.6, 0.8],
                       [0.3, 0.9],
                       [0.4, 0.7],
                       [0.5, 0.9],
                       [0.2, 0.9]])

summary = run_mc(M=200, N=500, I=5,
                 pi=0.4, alpha_true=alpha_true, p_true=p_true,
                 n_iter=3000, burn=1000, seed0=42)

print("\n95% posterior interval coverage (target ≈ 0.95)")
print("SE:", summary["coverage_se"].round(3))
print("SP:", summary["coverage_sp"].round(3))

print("\nBias of α (col-0 = α₍i0₎, col-1 = α₍i1₎):")
print(summary["bias_alpha"].round(4))

print("\nMSE of α:")
print(summary["mse_alpha"].round(5))


Monte-Carlo: 100%|██████████| 200/200 [05:54<00:00,  1.77s/it]


95% posterior interval coverage (target ≈ 0.95)
SE: [0.97  0.965 0.985 0.97  0.935]
SP: [0.945 0.975 0.97  0.97  0.96 ]

Bias of α (col-0 = α₍i0₎, col-1 = α₍i1₎):
[[-0.0372 -0.0303]
 [-0.0086 -0.0141]
 [-0.0102 -0.0253]
 [-0.0171 -0.0145]
 [-0.001  -0.02  ]]

MSE of α:
[[0.00291 0.00261]
 [0.00224 0.00207]
 [0.00287 0.00454]
 [0.00316 0.00167]
 [0.00188 0.00266]]
